**Import Library**

In [1]:
#--Import Library
import requests
from bs4 import BeautifulSoup as Soup
import pandas as pd
import numpy as np
import re
from datetime import datetime

#-----------------------------------------------------------------------------------------------------------------------------------------------------------

**Get URL Article from Pulsk.com**

In [2]:
#--Set URL Pulsk
root = "https://www.pulsk.com"
link = "/index/top"

In [125]:
#--Get Request to Pulsk Page
page = 3
url = requests.get(root+link+"/{}".format(str(page)))
request = url.text
soup_data = Soup(request, 'html.parser')

In [126]:
#--Mengambil semua tag artikel dalam satu halaman.
articel = soup_data.select('div.article')
print("Mendapatkan {} Artikel.".format(len(articel)))

Mendapatkan 50 Artikel.


In [127]:
#--Mengambil beberapa data dari masing" tag artikel.

Judul = []
Links = []
Times = []
Kategori = []

for x in articel:
    Judul.append(x.select_one('h4').text)
    Links.append(root+x.select_one('a').attrs['href'])
    Kategori.append(x.select_one('div.right').text)
    today = datetime.now()
    Times.append(today)
    
    print(" ")
    print("Kategori: ", x.select_one('div.right').text)
    print("="*111)
    print("Judul: ", x.select_one('h4').text)
    print("URL: ", root + x.select_one('a').attrs['href'])
    print("="*111)
    print("Time: ", today)
    print(" ")

 
Kategori:  Lucu Lucu
Judul:  Tanpa Disadari Apa yang Mereka Kenakan Mirip Banget Sama Keadaan Sekitar
URL:  https://www.pulsk.com/645419/
Time:  2022-01-27 10:35:18.328729
 
 
Kategori:  Beautiful / GIrl
Judul:  ORANG KOREA SEBELUM & SESUDAH OPERASI PLASTIK
URL:  https://www.pulsk.com/614425/
Time:  2022-01-27 10:35:18.331730
 
 
Kategori:  Lucu Lucu
Judul:  10 Kelakuan Random Emak-emak yang Bikin Tepuk Jidat
URL:  https://www.pulsk.com/662541/10-Kelakuan-Random-Emakemak-yang-Bikin-Tepuk-Jidat
Time:  2022-01-27 10:35:18.334731
 
 
Kategori:  Lucu Lucu
Judul:  10 Potret Gerobak Jualan yang Nggak Sesuai Dengan Dagangannya
URL:  https://www.pulsk.com/662360/10-Potret-Gerobak-Jualan-yang-Nggak-Sesuai-Dengan-Dagangannya
Time:  2022-01-27 10:35:18.335729
 
 
Kategori:  Lucu Lucu
Judul:  Ngakak! Gambar-gambar Lucu Tentang Kehidupan Cewek (Harapan dan Realita)
URL:  https://www.pulsk.com/635022/
Time:  2022-01-27 10:35:18.339728
 
 
Kategori:  Lucu Lucu
Judul:  10 Tulisan Karangan Bunga yang

In [129]:
#--Fix Bug Untuk Double Quote
def fix_bug_judul(xd):
    JudulN = []
    for x in xd:
        JudulN.append(x.replace('\'',''))
    return JudulN

Judul_New = fix_bug_judul(Judul) 

In [130]:
#--Merubah format datetime yang telah diambil sebelumnya.

Times_new_format = []

for x in Times: 
    str_time = datetime.strftime(Times[0], '%Y-%m-%d %H:%M:%S')
    Times_new_format.append(str_time)

print("Format Lama: ", Times[0])
print("Format Baru: ", Times_new_format[0])

Format Lama:  2022-01-27 10:35:18.328729
Format Baru:  2022-01-27 10:35:18


In [131]:
#--Menggabungkan semua data yang telah discrap sebelumnya.
data_scrap = list(zip(Judul_New,Kategori,Links,Times_new_format))

#-----------------------------------------------------------------------------------------------------------------------------------------------------------

**Save URL Article to Postgre Database**

In [132]:
#--Network Config
import psycopg2 as ps 

def connect_db():
    conn = ps.connect(database="postgres",
                    user="postgres",
                    password="admin",
                    host="localhost",
                    port=5432)
    return conn.cursor(), conn

In [133]:
#--Menyimpan data URL artikel kedalam table links_pulsk.
#--Data URL akan tersimpan jika data belum ada di database.

insert_script = "INSERT INTO links_pulsk (judul_head, kategori, url, time) VALUES (%s, %s, %s, %s)"
message = ["Query Fail! - Data is Exist!","Query Succes!"]

try:
    succes = 0
    fail = 0
    cur, conn = connect_db()
    for z in data_scrap:
        cur.execute("SELECT judul_head FROM links_pulsk WHERE judul_head = '{}'".format(z[0]))
        cek_exist = cur.fetchall()

        if cek_exist == []:        
            insert_value = (z[0], z[1], z[2], z[3])
            cur.execute(insert_script, insert_value)
            conn.commit()
            succes += 1
        else:
            fail +=1

except Exception as error:
    print(error)
    
finally:
    print(message[1])
    print("Succes Entry: {} Data".format(succes))
    print(" ")
    print(message[0])
    print("Failed Entry: {} Data".format(fail)) 
    conn.close()
        

Query Succes!
Succes Entry: 50 Data
 
Query Fail! - Data is Exist!
Failed Entry: 0 Data


#-----------------------------------------------------------------------------------------------------------------------------------------------------------

**Get URL Article from API**

In [134]:
#--Request Get to URL Article API with X-Api-Token 'dev'.

url_api = 'http://192.168.1.8:8001/article/get_article_url'
token = 'dev'

req = requests.get(url_api, 
    headers={'X-Api-Token': '{}'.format(token),
    }
)
req.raise_for_status()
data_url = req.json()
print("status:", req.status_code)

status: 200


In [135]:
#--Mengekstrak data URL artikel.
data_url = data_url['data']
data_url = data_url['url_list']
urls = pd.DataFrame(data_url)
urls.head()

,id_links,judul_head,kategori,url,time
0,153,10 Tulisan Typo Tugas Mahasiswa yang Bikin Dos...,Lucu Lucu,https://www.pulsk.com/661407/10-Tulisan-Typo-T...,2022-01-26
1,154,Momen-Momen Saat Lihat Benda Seperti Makanan I...,Lucu Lucu,https://www.pulsk.com/662952/MomenMomen-Saat-L...,2022-01-26
2,155,10 Foto Perpaduan Makanan yang Bikin Hilang Na...,Lucu Lucu,https://www.pulsk.com/662940/10-Foto-Perpaduan...,2022-01-26
3,156,8 Potret Barang Tiruan yang Bikin Ngakak,Lucu Lucu,https://www.pulsk.com/662000/8-Potret-Barang-T...,2022-01-26
4,157,10 Gaya Rambut Kocak yang Bikin Kamu Ketawa Sa...,Lucu Lucu,https://www.pulsk.com/654611/,2022-01-26


In [136]:
data_urls = urls.iloc[:,1:4]
data_urls.head()

,judul_head,kategori,url
0,10 Tulisan Typo Tugas Mahasiswa yang Bikin Dos...,Lucu Lucu,https://www.pulsk.com/661407/10-Tulisan-Typo-T...
1,Momen-Momen Saat Lihat Benda Seperti Makanan I...,Lucu Lucu,https://www.pulsk.com/662952/MomenMomen-Saat-L...
2,10 Foto Perpaduan Makanan yang Bikin Hilang Na...,Lucu Lucu,https://www.pulsk.com/662940/10-Foto-Perpaduan...
3,8 Potret Barang Tiruan yang Bikin Ngakak,Lucu Lucu,https://www.pulsk.com/662000/8-Potret-Barang-T...
4,10 Gaya Rambut Kocak yang Bikin Kamu Ketawa Sa...,Lucu Lucu,https://www.pulsk.com/654611/


#-----------------------------------------------------------------------------------------------------------------------------------------------------------

**Get Detail Article / Content Article by URL**

In [145]:
#--Get Request to URL Article.
articel_news = requests.get(data_urls.url[0])
request_news = articel_news.text
soup_data_news = Soup(request_news, 'html.parser')

In [146]:
#--Fungsi untuk mengambil isi konten dan konten gambar artikel.

def get_gambar_konten():
    kontens_list = []
    seps = ' - '
    konten = soup_data_news.select('img.images')
    for x in konten:
        kontens_list.append(x.attrs['src'])
    hasil = seps.join(kontens_list)
    return hasil

def get_konten():
    kontens_list = []
    sep = '. '
    seps = ' - '
    konten = soup_data_news.select('p.caption-album')
    for x in konten:
        stack_konten = []
        stack_konten.append(x.select_one('div.number-foto').text)
        kontens = re.sub("[0-9]","", x.text)
        stack_konten.append(kontens)
        kontens_list.append(sep.join(stack_konten))
    hasil = seps.join(kontens_list)
    return hasil


In [147]:
#--Testing Tag & Filtering Detail Article.
print("Judul: ", soup_data_news.select_one('div.title').text)
print("Penulis: ", soup_data_news.select_one('div.uname-detail').span.text)
print("Foto Profil: ", soup_data_news.select_one('div.posted-on').img.attrs['src'])
print("Thumbnail: ", soup_data_news.select_one('figure.detail-photo').img.attrs['src'])
print("Jumlah_Views: ", soup_data_news.select_one('div.views.right').text.replace(',','')[:-6])
print("Deskripsi: ", soup_data_news.select_one('p').text)
kontens_list = get_konten()
gambars_list = get_gambar_konten()
print("Konten: ", kontens_list)
print("Konten Gambar: ", gambars_list)
print("Tanggal: ", soup_data_news.select_one('div.uname-detail').time.attrs['datetime'])

Judul:  10 Tulisan Typo Tugas Mahasiswa yang Bikin Dosen Gagal Paham
Penulis:  Jenifer Sun
Foto Profil:  https://www3.pictures.zimbio.com/gi/Jennifer+Lawrence+2017+Toronto+International+G0c6loNNtHjl.jpg
Thumbnail:  https://www.pulsk.com/images/2020/10/27/16038155135f984859dbb1f.jpg
Jumlah_Views:  12602
Deskripsi:  Masa kuliah adalah masa dimana kita mulai beranjak dewasa. Di mana kita bertemu dengan orang-orang yang baru dan nggak bisa lagi bersantai-santai seperti ketika kita sekolah dulu. Apapun yang kita lakukan pasti nggak jauh dari yang namanya tugas. Kalau kita nggak fokus saat kuliah, bisa dijamin kita pasti ketinggalan dari yang lain.Hal terberat yang harus dihadapi oleh mahasiswa adalah ketika menghadapi tugas akhir dan harus mengerjakan skripsi. Deadline waktu yang membuat kita harus fokus mengerjakannya. Sedikit saja nggak fokus, bisa-bisa tugas yang dikerjakan akan berantakan.Sama seperti beberapa mahasiswa yang mengerjakan tugas berikut ini. Mungkin karena mengantuk atau k

In [148]:
#--Mengambil semua data terkait konten atau detail dari artikel.
#--Pengambilan dilakukan berdasarkan url artikel yang telah didapatkan sebelumnya.

Judul = []
Penulis = []
Foto_Profil = []
Kategori = []
Thumbnail = []
Jumlah_Views = []
Deskripsi = []
Konten = []
Konten_Gambar = []
Tanggal = []
kate = 0

for x in data_urls.url:
    articel_news = requests.get(x)
    request_news = articel_news.text
    soup_data_news = Soup(request_news, 'html.parser')

    Judul.append(soup_data_news.select_one('div.title').text)
    Penulis.append(soup_data_news.select_one('div.uname-detail').span.text)
    Foto_Profil.append(soup_data_news.select_one('div.posted-on').img.attrs['src'])
    Kategori.append(data_urls.kategori[kate])
    Thumbnail.append(soup_data_news.select_one('figure.detail-photo').img.attrs['src'])
    Jumlah_Views.append(soup_data_news.select_one('div.views.right').text.replace(',','')[:-6])
    Deskripsi.append(soup_data_news.select_one('p').text)
    kontens_list = get_konten()
    gambars_list = get_gambar_konten()
    Konten.append(kontens_list)
    Konten_Gambar.append(gambars_list)
    Tanggal.append(soup_data_news.select_one('div.uname-detail').time.attrs['datetime'])
    kate += 1


In [151]:
#--Menggabungkan semua data artikel yang telah discrap sebelumnya.
Judul_New = fix_bug_judul(Judul)
data_scrap_article = list(zip(Judul_New,Penulis,Foto_Profil,Kategori,Thumbnail,Jumlah_Views,Deskripsi,Konten,Konten_Gambar,Tanggal))

#-----------------------------------------------------------------------------------------------------------------------------------------------------------

**Save Detail Article / Content Article If Not Exist in Database**

In [152]:
#--Save Detail Article / Content Article If Not Exist in Database
inserts_script = """INSERT INTO data_scrap_pulsk (judul, penulis, foto_profil, kategori, thumbnail, jumlah_views, deskripsi, konten, konten_gambar, tanggal)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

try:
    succes = 0
    fail = 0
    cur, conn = connect_db()
    for z in data_scrap_article:
        cur.execute("SELECT judul FROM data_scrap_pulsk WHERE judul = '{}'".format(z[0]))
        cek_exist = cur.fetchall()

        if cek_exist == []:        
            inserts_value = (z[0], z[1], z[2], z[3], z[4], z[5], z[6], z[7], z[8], z[9])
            cur.execute(inserts_script, inserts_value)
            conn.commit()
            succes += 1
        else:
            fail +=1

except Exception as error:
    print(error)
    
finally:
    print(message[1])
    print("Succes Entry: {} Data".format(succes))
    print(" ")
    print(message[0])
    print("Failed Entry: {} Data".format(fail)) 
    conn.close()

Query Succes!
Succes Entry: 0 Data
 
Query Fail! - Data is Exist!
Failed Entry: 150 Data


**Get All Detail Article / Content Article using Article API**

In [36]:
#--Request Get to URL Article API with X-Api-Token 'dev'.

url_api = 'http://localhost:8001/article/get_detail_article'
token = 'dev'

req = requests.get(url_api, 
    headers={'X-Api-Token': '{}'.format(token),
    }
)
req.raise_for_status()
data_article = req.json()
print("status:", req.status_code)

status: 200


In [40]:
#--Mengekstrak data detail artikel / konten artikel.
data_art = data_article['data']
data_art = data_art['scrap_list']
data_art = pd.DataFrame(data_art, columns=['id_scrap',
                                            'judul',
                                            'penulis',
                                            'foto_profil',
                                            'kategori',
                                            'thumbnail',
                                            'jumlah_views',
                                            'deskripsi',
                                            'konten',
                                            'konten_gambar',
                                            'tanggal'])
data_art.head()

,id_scrap,judul,penulis,foto_profil,kategori,thumbnail,jumlah_views,deskripsi,konten,konten_gambar,tanggal
0,1001,10 Tulisan Typo Tugas Mahasiswa yang Bikin Dos...,Jenifer Sun,https://www3.pictures.zimbio.com/gi/Jennifer+L...,Lucu Lucu,https://www.pulsk.com/images/2020/10/27/160381...,12553,Masa kuliah adalah masa dimana kita mulai bera...,"1. Fauzan, buruk banget nasibmu :D - 2. Antara...",https://www.pulsk.com/images/2020/10/27/160381...,2020-10-27
1,1002,Momen-Momen Saat Lihat Benda Seperti Makanan I...,Robert Giovani,https://encrypted-tbn0.gstatic.com/images?q=tb...,Lucu Lucu,https://www.pulsk.com/images/2021/12/22/164014...,3907,Kalau udah masalah perut memang nggak bisa dia...,"1. Jangan salah ambil makanan ya gaes, bahaya....",https://www.pulsk.com/images/2021/12/22/164014...,2021-12-22
2,1003,10 Foto Perpaduan Makanan yang Bikin Hilang Na...,Siska Komo,https://s2.bukalapak.com/img/7411869191/large/...,Lucu Lucu,https://www.pulsk.com/images/2021/12/19/163988...,2297,Setiap hari kita harus memakan makanan agar me...,"1. Makan mie sana nasi aja udah aneh, ini maka...",https://www.pulsk.com/images/2021/12/19/163988...,2021-12-19
3,1004,8 Potret Barang Tiruan yang Bikin Ngakak,Astro Maya,https://vignette.wikia.nocookie.net/aroha/imag...,Lucu Lucu,https://www.pulsk.com/images/2021/03/30/161710...,7646,Barang original biasanya terbukti berbahan bag...,1. Bisa banget ada barang KW di supermarket. M...,https://www.pulsk.com/images/2021/03/30/161710...,2021-03-30
4,1005,10 Gaya Rambut Kocak yang Bikin Kamu Ketawa Sa...,Ochaaa,http://static.pulsk.com/images/2017/01/11/1484...,Lucu Lucu,https://www.pulsk.com/images/2017/12/18/151358...,37532,Bagi semua orang rambut adalah mahkota. Jadi n...,"1. Merasa bukan anak kecil lagi, anak ini memi...",https://www.pulsk.com/images/2017/12/18/151358...,2017-12-18
